# Sentiment Analysis - Part 3: Dataset Preprocessing, Model Training and Fine-Tuning (BERT)

Dataset: https://data.mendeley.com/datasets/z9zw7nt5h2/1

In [1]:
# Import Libraries
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

c:\Users\mstoe\anaconda3\envs\pytorch_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset

In [3]:
# Load dataset
dataset = load_dataset("SetFit/sst5")

Repo card metadata block was not found. Setting CardData to empty.


In [4]:
train_dataset = dataset['train']

In [5]:
# Load the Pre-trained BERT Tokenizer and Model
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)  # Will be trained later

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Tokenize the dataset
def tokenize_function(data):
    return tokenizer(data['text'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 2210/2210 [00:00<00:00, 2378.99 examples/s]


In [7]:
# Prepare Data for Training
train_dataset = tokenized_datasets['train']
eval_dataset = tokenized_datasets['validation']
test_dataset = tokenized_datasets['test']

In [8]:
# Define Compute Metrics Function

def compute_metrics(p):
    preds = p.predictions.argmax(axis=-1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [9]:
# Set up Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
)

In [10]:
# Initialize The Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

: 

In [ ]:
# Train the Model
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
# Evaluate the Model
results = trainer.evaluate(test_dataset)
results

In [ ]:
# Save the Model and Tokenizer
model.save_pretrained('./sentiment_model')
tokenizer.save_pretrained('./sentiment_model')

In [ ]:
# Load the Model and Tokenizer for Inference
model = AutoModelForSequenceClassification.from_pretrained('./sentiment_model')
tokenizer = AutoTokenizer.from_pretrained('./sentiment_model')

In [ ]:
# Make Predictions
def predict(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax(dim=-1).item()
    return predicted_class

# Example usage
text = ''

prediction = predict(text)
print(f"Predicted sentiment class: {prediction}")